## Problem 2

For 1st Part, a Convolution Neural Network for all 10 classes in CIFAR-10 will be created. The fully connected layer will be adjusted at the end with respect to the number of output classes.The Network will be trained for 300 epochs. Training Time, Training Loss, and Evaluation Accuracy after 300 epochs will be reported.

## Problem 2, Part a

In [1]:
# Importing all the necessary Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Importing Pytorch and Tensorflow
import torch
torch.version.__version__
import torch.optim as optim
import torch.nn as nn

In [20]:
# Importing/Downloading the CIFAR-10 DataSet
from torchvision import datasets
data_path = "../data-unversioned/p1ch7/"

# Convertion of the PIL Images into PyTorch Tensors
from torchvision import transforms

tensor_cifar10 = datasets.CIFAR10(data_path, train = True, download = True,
                                  transform = transforms.Compose([transforms.ToTensor(),
                                              transforms.Normalize((0.4914, 0.4822, 0.4465),
                                                                   (0.2470, 0.2435, 0.2616))]))
tensor_cifar10_val = datasets.CIFAR10(data_path, train = False, download = True,
                                      transform = transforms.Compose([transforms.ToTensor(),
                                                 transforms.Normalize((0.4942, 0.4851, 0.4404),
                                                                      (0.2467, 0.2429, 0.2616))]))



Files already downloaded and verified
Files already downloaded and verified


In [8]:
#Importing necessary Libraries for CNN 
import torch.nn.functional as F
import datetime

In [9]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(8 * 8 * 8, 32),
            nn.Tanh(),
            nn.Linear(32, 10))


In [10]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
    self.act1 = nn.Tanh()
    self.pool1 = nn.MaxPool2d(2)
    self.conv2 = nn.Conv2d(16,8, kernel_size=3, padding=1)
    self.act2 = nn.Tanh()
    self.pool2 = nn.MaxPool2d(2)
    self.fc1 = nn.Linear(8*8*8,32)
    self.act3 = nn.Tanh()
    self.fc2 = nn.Linear(32,10)

  def forward(self, x):
    out = self.pool1(self.act1(self.conv1(x)))
    out = self.pool2(self.act2(self.conv2(out)))
    out = out.view(-1, 8*8*8)
    out = self.act3(self.fc1(out))
    out = self.fc2(out)
    return out


In [12]:
# Defining the Training Loop
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs +1):
        loss_train = 0.0;
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
        if(epoch == 1 or epoch % 10 == 0):
            print("{} Epoch {}, Training Loss {}".format(datetime.datetime.now(), epoch, loss_train / len(train_loader)))

In [17]:
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size = 64, shuffle = True)

# Training Data with Data Loader
model = Net()
optimizer = optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs = 300, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

2022-12-10 12:30:52.997731 Epoch 1, Training Loss 2.0543598640910194
2022-12-10 12:36:13.546336 Epoch 10, Training Loss 1.1958040703288124
2022-12-10 12:42:05.820028 Epoch 20, Training Loss 1.013959146292923
2022-12-10 12:47:52.256805 Epoch 30, Training Loss 0.9064702283390953
2022-12-10 12:53:40.274262 Epoch 40, Training Loss 0.843773039786712
2022-12-10 12:59:23.852780 Epoch 50, Training Loss 0.8058790556152763
2022-12-10 13:06:02.444425 Epoch 60, Training Loss 0.7749857910148933
2022-12-10 13:12:38.107592 Epoch 70, Training Loss 0.747799181991526
2022-12-10 13:18:37.942449 Epoch 80, Training Loss 0.729450122932034
2022-12-10 13:24:24.360593 Epoch 90, Training Loss 0.7102556972933547
2022-12-10 13:30:09.199887 Epoch 100, Training Loss 0.692855961754194
2022-12-10 13:35:59.027673 Epoch 110, Training Loss 0.6802598592799033
2022-12-10 13:41:46.741818 Epoch 120, Training Loss 0.6655139482920737
2022-12-10 13:47:34.024318 Epoch 130, Training Loss 0.6543021775648722
2022-12-10 13:53:21.46

In [22]:
# Measuring the Accuracy of the Trained Model

val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64, shuffle=False)


def validate(model, train_loader, val_loader):
    for name, loader in[("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.2f}".format(name, (correct/total)))

validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.62


## Problem 2, Part b

In [24]:
model = nn.Sequential(
            nn.Conv2d(3,16, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(16,8, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Conv2d(8,4, kernel_size=3, padding=1),
            nn.Tanh(),
            nn.MaxPool2d(2),
            nn.Linear(4*4*4, 32),
            nn.Tanh(),
            nn.Linear(32, 10))

In [25]:
train_loader = torch.utils.data.DataLoader(tensor_cifar10, batch_size = 64, shuffle = True)

# Training Data with Data Loader
model = Net()
optimizer = optim.SGD(model.parameters(), lr = 1e-2)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs = 300, optimizer = optimizer, model = model, loss_fn = loss_fn, train_loader = train_loader)

2022-12-10 17:27:34.303518 Epoch 1, Training Loss 2.0369032587846525
2022-12-10 17:33:11.410521 Epoch 10, Training Loss 1.1831883990856082
2022-12-10 17:38:58.658766 Epoch 20, Training Loss 1.0020822544232049
2022-12-10 17:44:46.279317 Epoch 30, Training Loss 0.9225083262566716
2022-12-10 17:50:37.155050 Epoch 40, Training Loss 0.8689730754669975
2022-12-10 17:56:26.426281 Epoch 50, Training Loss 0.8279266221748899
2022-12-10 18:02:13.856025 Epoch 60, Training Loss 0.7939400663003897
2022-12-10 18:47:52.085517 Epoch 70, Training Loss 0.7667722842653694
2022-12-10 18:57:10.743248 Epoch 80, Training Loss 0.742296373181026
2022-12-10 19:06:35.121050 Epoch 90, Training Loss 0.7218191839011429
2022-12-10 19:15:41.094067 Epoch 100, Training Loss 0.7036517176329328
2022-12-10 19:24:34.005317 Epoch 110, Training Loss 0.6875542137019165
2022-12-10 19:33:32.481298 Epoch 120, Training Loss 0.6743000709187345
2022-12-10 19:42:39.400717 Epoch 130, Training Loss 0.6621979605739988
2022-12-10 19:51:4

In [26]:
# Measuring the Accuracy of the Trained Model

val_loader = torch.utils.data.DataLoader(tensor_cifar10_val, batch_size=64, shuffle=False)


def validate(model, train_loader, val_loader):
    for name, loader in[("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0
        
        with torch.no_grad():
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim = 1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())
        print("Accuracy {}: {:.2f}".format(name, (correct/total)))

validate(model, train_loader, val_loader)

Accuracy train: 0.80
Accuracy val: 0.62
